<a href="https://colab.research.google.com/github/rdenadai/sentiment-analysis-2018-president-election/blob/edgarbanhesse/testes_valencia_projeto_final_ia369y_2sem_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testes de valência para o projeto final de IA369Y 2 Semestre 2018

Passos para tratar os dados com valência, testar e escolher um classificador para utilizar no projeto final de IA369Y.

1) Remover espaços duplos, quebras de linha, números e links do dataset e das frases a serem testadas.

2) Remover stopwords e aplicar o stemmer.

3) Treinar os classificadores.

4) Realizar as predições com os classificadores.

5) Avaliar as medidas obtidas com os classificadores.

In [0]:
#Importação das bibliotecas para testes
import csv
import codecs
import copy
import re
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
#Instalação e download
!pip install emoji
!python -m spacy download pt

import time
import concurrent.futures
from unicodedata import normalize
from string import punctuation
from functools import lru_cache

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import emoji

nltk.download('stopwords')

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')


def _load_emotion_file_content(emotion, path='dataset/emocoes'):
    with open(f'{path}/{emotion}', 'r') as h:
        words = h.readlines()
        for i, word in enumerate(words):
            word = word.replace('\n', '').lower().strip()
            words[i] = STEMMER.stem(word)
            # words[i] = [w.lemma_ for w in NLP(word, disable=['parser'])][0]
    return sorted(list(set(words)))


@lru_cache(maxsize=256)
def load_six_emotions(filepath):
    """Ekman, Friesen, and Ellsworth : anger, disgust, fear, joy, sadness, surprise."""
    emotion_words = {
        'ALEGRIA': _load_emotion_file_content('alegria', filepath),
        'DESGOSTO': _load_emotion_file_content('desgosto', filepath),
        'MEDO': _load_emotion_file_content('medo', filepath),
        'RAIVA': _load_emotion_file_content('raiva', filepath),
        'SURPRESA': _load_emotion_file_content('surpresa', filepath),
        'TRISTEZA': _load_emotion_file_content('tristeza', filepath),
    }
    return emotion_words


@lru_cache(maxsize=256)
def load_3_emotions(filepath):
    """Ekman, Friesen, and Ellsworth : anger, disgust, fear, joy, sadness, surprise."""
    emotion_words = {
        'POSITIVO': _load_emotion_file_content('positivo', filepath),
        'NEGATIVO': _load_emotion_file_content('negativo', filepath),
        'NEUTRO': _load_emotion_file_content('neutro', filepath),
    }
    return emotion_words


@lru_cache(maxsize=256)
def load_valence_emotions(filename_oplexicon, filename_sentilex):
    data = {
        'POSITIVO': [],
        'NEGATIVO': [],
        'NEUTRO': [],
    }

    oplexicon = load_valence_emotions_from_oplexicon(filename_oplexicon)
    sentilex = load_valence_emotions_from_sentilex(filename_sentilex)

    data['POSITIVO'] = oplexicon['POSITIVO'] + sentilex['POSITIVO']
    data['NEGATIVO'] = oplexicon['NEGATIVO'] + sentilex['NEGATIVO']
    data['NEUTRO'] = oplexicon['NEUTRO'] + sentilex['NEUTRO']
    data['POSITIVO'] = sorted(list(set(data['POSITIVO'])))
    data['NEGATIVO'] = sorted(list(set(data['NEGATIVO'])))
    data['NEUTRO'] = sorted(list(set(data['NEUTRO'])))
    return data


@lru_cache(maxsize=256)
def load_valence_emotions_from_oplexicon(filename):
    """NEUTRAL | POSITIVE | NEGATIVE."""
    spacy_conv = {
        'adj': 'ADJ',
        'n': 'NOUN',
        'vb': 'VERB',
        'det': 'DET',
        'emot': 'EMOT',
        'htag': 'HTAG'
    }

    data = {
        'POSITIVO': [],
        'NEGATIVO': [],
        'NEUTRO': [],
    }
    with codecs.open(filename, 'r', 'UTF-8') as hf:
        lines = hf.readlines()
        for line in lines:
            info = line.lower().split(',')
            if len(info[0].split()) <= 1:
                info[1] = [spacy_conv.get(tag) for tag in info[1].split()]
                word, tags, sent = info[:3]
                if 'HTAG' not in tags and 'EMOT' not in tags:
                    word = STEMMER.stem(word.lower().strip())
                    # word = [w.lemma_ for w in NLP(word.lower().strip(), disable=['parser'])][0]
                    if len(word) > 2:
                        sent = int(sent)
                        if sent == 1:
                            data['POSITIVO'] += [word]
                        elif sent == -1:
                            data['NEGATIVO'] += [word]
                        else:
                            data['NEUTRO'] += [word]
    data['POSITIVO'] = sorted(list(set(data['POSITIVO'])))
    data['NEGATIVO'] = sorted(list(set(data['NEGATIVO'])))
    data['NEUTRO'] = sorted(list(set(data['NEUTRO'])))
    return data


@lru_cache(maxsize=256)
def load_valence_emotions_from_sentilex(filename):
    """NEUTRAL | POSITIVE | NEGATIVE."""
    data = {
        'POSITIVO': [],
        'NEGATIVO': [],
        'NEUTRO': [],
    }
    with codecs.open(filename, 'r', 'UTF-8') as hf:
        lines = hf.readlines()
        for line in lines:
            info = line.lower().split('.')
            words = [word.strip() for word in info[0].split(',')]
            for word in words:
                word = STEMMER.stem(word.lower().strip())
                # word = [w.lemma_ for w in NLP(word.lower().strip(), disable=['parser'])][0]
                if len(word) > 2:
                    cdata = info[1].split(';')
                    if len(cdata) > 0:
                        sent0 = [int(k.replace('pol:n0=', '')) if 'pol:n0=' in k else None for k in cdata]
                        sent1 = [int(k.replace('pol:n1=', '')) if 'pol:n1=' in k else None for k in cdata]
                        sent0 = list(filter(None.__ne__, sent0))
                        sent1 = list(filter(None.__ne__, sent1))
                        if len(sent0) >= 1 and len(sent1) <= 0:
                            sent = sent0[0]
                            if sent == 1:
                                data['POSITIVO'] += [word]
                            elif sent == -1:
                                data['NEGATIVO'] += [word]
                            else:
                                data['NEUTRO'] += [word]
    data['POSITIVO'] = sorted(list(set(data['POSITIVO'])))
    data['NEGATIVO'] = sorted(list(set(data['NEGATIVO'])))
    data['NEUTRO'] = sorted(list(set(data['NEUTRO'])))
    return data


@lru_cache(maxsize=256)
def is_number(s):
    try:
        complex(s) # for int, long, float and complex
    except ValueError:
        return False
    return True


@lru_cache(maxsize=256)
def _get_stopwords():
    stpwords = stopwords.words('portuguese')
    rms = ['um', 'não', 'mais', 'muito']
    for rm in rms:
        del stpwords[stpwords.index(rm)]
    return stpwords, punctuation


def generate_corpus(documents=None, debug=False):
    assert len(documents) > 0
    if debug: print('Iniciando processamento...')
    tokenized_docs = documents
    with concurrent.futures.ProcessPoolExecutor() as procs:
        if debug: print('Executando processo de remoção das stopwords...')
        tokenized_frases = procs.map(tokenizer, tokenized_docs, chunksize=25)
    if debug: print('Finalizado...')
    return list(tokenized_frases)


def tokenizer(phrase, clean=False):
    if not clean:
        phrase = clean_up(phrase)
    phrase = NLP(phrase, disable=['parser'])
    clean_frase = []
    clfa = clean_frase.append
    for palavra in phrase:
        if palavra.pos_ != 'PUNCT':
            word = palavra.text.strip()
            if not is_number(word) and len(word) > 1:
                clfa(STEMMER.stem(palavra.text))
    return ' '.join(clean_frase)


def clean_up(phrase):
    STOPWORDS, PUNCT = _get_stopwords()
    # Transforma as hashtags em palavras
    try:
        for group in re.findall(r'#\S+\b', phrase, re.DOTALL):
            g2 = re.sub(r'([A-Z])', r' \1', group, flags=re.MULTILINE)
            phrase = re.sub(r'{}\b'.format(group), g2, phrase, flags=re.MULTILINE)
    except Exception:
        pass
    # lowercase para fazer outros pré-processamentos
    phrase = phrase.lower()
    phrase = emoji.get_emoji_regexp().sub(r'', phrase)
    for stw in STOPWORDS:
        phrase = re.sub(r'\b{}\b'.format(stw), '', phrase, flags=re.MULTILINE)
    for punct in PUNCT:
        phrase = phrase.replace(punct, ' ')
    for o, r in RM:
        phrase = re.sub(o, r, phrase, flags=re.MULTILINE)
    return phrase


# GLOBALS
NLP = spacy.load('pt')
# STEMMER = nltk.stem.RSLPStemmer()
STEMMER = nltk.stem.SnowballStemmer('portuguese')
STOPWORDS, PUNCT = _get_stopwords()
RM = [
    (r'\n+', r' . '), (r'"', r' '), (r'\'', r' '),  (r'@', r''), (r'[…]', ' . '), (r'[0-9]*', r''),
    (r'#', r''), (r'(RT)', r''), (r'(http[s]*?:\/\/)+.*[\r\n]*', r''),
    (r'“', r''), (r'”', ''), (r'([aeiouqwtyupdfghjklçzxcvbnm|!@$%&\.\[\]\(\)+-_=<>,;:])\1+', r'\1'),
    (r'(ñ)', r'não'), (r'(nã)', r'não'), (r'\s+', r' '),
]

    100% |████████████████████████████████| 38.7MB 75.5MB/s 
  Running setup.py install for pt-core-news-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/pt

    You can now load the model via spacy.load('pt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Datasets

Para validar, serão utilizados dois datasets.

Os dos datasets foram obtidos do site minerando dados.

O primeiro deles tem tweets de política de Minas Gerais com rótulos de valência: positivo, negativo e neutro. Foi feito um tratamento para eliminar tweets repetidos e dessa forma sobraram 3016 tweets.

O segundo contém 2123 títulos de notícias com rótulos de valência: positivo, negativo e neutro.

In [3]:
#Download dos datasets
!ls -la
!rm -f *.csv
!rm -f *.txt
!wget https://raw.githubusercontent.com/rdenadai/sentiment-analysis-2018-president-election/edgarbanhesse/material-apoio/tweets_mg_tratados.csv
!wget https://raw.githubusercontent.com/rdenadai/sentiment-analysis-2018-president-election/edgarbanhesse/material-apoio/titulo_noticias.txt
!wget https://raw.githubusercontent.com/rdenadai/sentiment-analysis-2018-president-election/edgarbanhesse/material-apoio/50_tweets_mg.csv
!ls -la

total 16
drwxr-xr-x 4 root root 4096 Nov 15 01:02 .
drwxr-xr-x 1 root root 4096 Nov 15 18:23 ..
drwxr-xr-x 4 root root 4096 Nov 15 00:52 .config
drwxr-xr-x 2 root root 4096 Nov 15 01:02 sample_data
--2018-11-15 18:59:24--  https://raw.githubusercontent.com/rdenadai/sentiment-analysis-2018-president-election/edgarbanhesse/material-apoio/tweets_mg_tratados.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 295061 (288K) [text/plain]
Saving to: ‘tweets_mg_tratados.csv’

tweets_mg_tratados. 100%[===================>] 288.15K  --.-KB/s    in 0.03s   

2018-11-15 18:59:24 (8.96 MB/s) - ‘tweets_mg_tratados.csv’ saved [295061/295061]

--2018-11-15 18:59:25--  https://raw.githubusercontent.com/rdenadai/sentiment-analysis-2018-president-election/edgarbanhesse/material-apo

In [4]:
#Carregando os datasets
def carregar(filename):
    frases = []
    with open(filename, 'r', encoding='utf-8') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = tokenizer(row[0]).strip()
            valencia = row[1].upper()
            if len(frase) > 5:
                frases.append((valencia, frase))
    return frases

frases = carregar('tweets_mg_tratados.csv')
frases += carregar('titulo_noticias.txt')

shuffle(frases)

print(frases[:15])

[('NEGATIVO', 'renan ped nom cpi mist petrobr cri cpi mist metrô notíc polít'), ('POSITIVO', 'açõ roub cen abertur pregã'), ('NEUTRO', 'lucianal mir quier ni imagin ese estad así mg'), ('NEGATIVO', 'tip mg skol ruim brahm boa aqu contrári isso'), ('NEUTRO', 'rt fnk pimentel torr dinheir depósit judic pud entreg tod dinheir mã esse corrupt'), ('POSITIVO', 'sesc inscriçõ abert curs tod estad'), ('POSITIVO', 'açã governomg benefic mil agricultor famili'), ('NEUTRO', 'aproxim bilhã pagament resgat depósit judic sum cont govern min ger'), ('NEUTRO', 'rt edvaldomoraisp estad min violênc avanc interior min'), ('POSITIVO', 'pmg vespasian pm aprend armas drog'), ('POSITIVO', 'faculdad vag curs gratuit vár áre'), ('NEUTRO', 'blog esmael justic eleitoral proíb uso máquin eleiçõ mand rich retir fot sit copel sanep'), ('NEGATIVO', 'açõ balanc abertur pregã segund feir'), ('POSITIVO', 'rt felaci presídi min adot nov model conseg recuper pres'), ('NEGATIVO', 'rt rbrasiliens calam financeir govern mg 

In [5]:
#Carrega os datasets em separado
tweets_mg = []
titulo_noticias = []

tweets_mg = carregar('tweets_mg_tratados.csv')
titulo_noticias = carregar('titulo_noticias.txt')

print(tweets_mg)
print(titulo_noticias)

[('NEUTRO', 'bom band mort'), ('NEUTRO', 'fóruns region govern vã eleg nov prefeit vereador'), ('NEGATIVO', 'govern min ger compr mais dois helicópter'), ('POSITIVO', 'polic milit faz prisõ aprend armas fog drog bail funk'), ('POSITIVO', 'um cab políc milit pm anos folg imped um roub pad noit esta'), ('NEUTRO', 'medíocr'), ('NEGATIVO', 'pedr vc acha comunic govern govern'), ('NEGATIVO', 'sulfúr eh dirí soy estúp qué verguenz'), ('NEGATIVO', 'che conoc nov'), ('NEUTRO', 'mineir diz não torc tim nenhum dentr um estad atlét mg cruzeir pq'), ('POSITIVO', 'minsaud envi equip forc nacional sus auxili atend pacient suspeit febr amarel mg colet'), ('NEUTRO', 'fal sobr seman vem diz dor sobr tarif transport nacional estad min'), ('NEGATIVO', 'míd mund tod arranc vár cabec comemor funk facçã nacional estad min'), ('NEUTRO', 'míd mund tod arranc vár cabec comemor funk'), ('NEGATIVO', 'jeitinh kalil nov prefeit bh almoc marmitex elimin traj formal polit estad min'), ('NEUTRO', 'mg cultur dic cultu

In [6]:
#all_datasets
afrases = []
avalencias = []
for valencia, frase in frases:
    afrases.append(frase)
    avalencias.append(valencia)
    
print(afrases)
print(avalencias)


#tweets_mg
atweets_mg = []
aval_tweets_mg = []
for valencia, frase in tweets_mg:
    atweets_mg.append(frase)
    aval_tweets_mg.append(valencia)

print(atweets_mg)
print(aval_tweets_mg)


#tweets_mg
atitulo_noticias = []
aval_titulo_noticias = []
for valencia, frase in titulo_noticias:
    atitulo_noticias.append(frase)
    aval_titulo_noticias.append(valencia)

print(atitulo_noticias)
print(aval_titulo_noticias)

['renan ped nom cpi mist petrobr cri cpi mist metrô notíc polít', 'açõ roub cen abertur pregã', 'lucianal mir quier ni imagin ese estad así mg', 'tip mg skol ruim brahm boa aqu contrári isso', 'rt fnk pimentel torr dinheir depósit judic pud entreg tod dinheir mã esse corrupt', 'sesc inscriçõ abert curs tod estad', 'açã governomg benefic mil agricultor famili', 'aproxim bilhã pagament resgat depósit judic sum cont govern min ger', 'rt edvaldomoraisp estad min violênc avanc interior min', 'pmg vespasian pm aprend armas drog', 'faculdad vag curs gratuit vár áre', 'blog esmael justic eleitoral proíb uso máquin eleiçõ mand rich retir fot sit copel sanep', 'açõ balanc abertur pregã segund feir', 'rt felaci presídi min adot nov model conseg recuper pres', 'rt rbrasiliens calam financeir govern mg compr dois helicópter', 'cartacapital tud um absurd legal aqu', 'souz cruz mult mil', 'rt portaldoholand presídi adot nov métod conseg recuper pres', 'bovesp índic ibovesp abre pratic estável sext ju

In [0]:
def run_ml_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    print(f'Modelo   : {model.__class__.__name__}')
    print(f'Acurácia : {np.round(model.score(X_test, y_test) * 100, 2)}%')
    print('-' * 20)

def split_data(X, y):
    test_size = .3
    random_state = 0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }

## Classificadores

In [0]:
classifiers = (
    LogisticRegression(multi_class='auto', solver='lbfgs'),
    RandomForestClassifier(n_estimators=20),
    LinearSVC(max_iter=1500),
    MultinomialNB(),
    KNeighborsClassifier(),
    MLPClassifier(max_iter=500),
    SVC(gamma='auto', max_iter=1500),
)

## TF-IDF

In [0]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

vec_tfidf_tmg = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tmg = vec_tfidf_tmg.fit_transform(atweets_mg)

vec_tfidf_tn = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tn = vec_tfidf_tn.fit_transform(atitulo_noticias)

In [16]:
print("\nall_datasets")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_tfidf, avalencias))
    except:
        pass

print("\ntweets_mg")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_tfidf_tmg, aval_tweets_mg))
    except:
        pass

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_tfidf_tn, aval_titulo_noticias))
    except:
        pass      


all_datasets
Modelo   : RandomForestClassifier
Acurácia : 62.68%
--------------------
Modelo   : LinearSVC
Acurácia : 66.19%
--------------------
Modelo   : MultinomialNB
Acurácia : 58.19%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 60.86%
--------------------
Modelo   : MLPClassifier
Acurácia : 64.5%
--------------------
Modelo   : SVC
Acurácia : 42.72%
--------------------

tweets_mg
Modelo   : RandomForestClassifier
Acurácia : 62.71%
--------------------
Modelo   : LinearSVC
Acurácia : 63.37%
--------------------
Modelo   : MultinomialNB
Acurácia : 62.49%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 58.05%
--------------------
Modelo   : MLPClassifier
Acurácia : 61.49%
--------------------
Modelo   : SVC
Acurácia : 43.51%
--------------------

titulo_noticias
Modelo   : RandomForestClassifier
Acurácia : 61.85%
--------------------
Modelo   : LinearSVC
Acurácia : 67.5%
--------------------
Modelo   : MultinomialNB
Acurácia : 61.85%
------------

## LSA (usando TF-IDF)

In [17]:
#all_datasets
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_svd, avalencias))
    except:
        pass


#tweets_mg
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
    except:
        pass


#titulo_noticias
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
    except:
        pass


all_datasets
Modelo   : RandomForestClassifier
Acurácia : 48.11%
--------------------
Modelo   : LinearSVC
Acurácia : 48.83%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 49.61%
--------------------
Modelo   : MLPClassifier
Acurácia : 51.95%
--------------------
Modelo   : SVC
Acurácia : 44.67%
--------------------

tweets_mg
Modelo   : RandomForestClassifier
Acurácia : 57.27%
--------------------
Modelo   : LinearSVC
Acurácia : 57.49%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 56.71%
--------------------
Modelo   : MLPClassifier
Acurácia : 58.82%
--------------------
Modelo   : SVC
Acurácia : 54.72%
--------------------

titulo_noticias
Modelo   : RandomForestClassifier
Acurácia : 50.71%
--------------------
Modelo   : LinearSVC
Acurácia : 51.81%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 48.35%
--------------------
Modelo   : MLPClassifier
Acurácia : 49.29%
--------------------
Modelo   : SVC
Acurácia : 46.78%
-------------

## LDA (usando TF-IDF)¶

In [18]:
#all_datasets
lda = LatentDirichletAllocation(n_components=5, random_state=0)
X_lda = lda.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_lda, avalencias))
    except:
        pass


#tweets_mg
lda = LatentDirichletAllocation(n_components=5, random_state=0)
X_lda = lda.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
    except:
        pass


#titulo_noticias
lda = LatentDirichletAllocation(n_components=5, random_state=0)
X_lda = lda.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
    except:
        pass      

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)



all_datasets
Modelo   : RandomForestClassifier
Acurácia : 45.25%
--------------------
Modelo   : LinearSVC
Acurácia : 47.33%
--------------------
Modelo   : MultinomialNB
Acurácia : 42.72%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 43.5%
--------------------
Modelo   : MLPClassifier
Acurácia : 47.46%
--------------------


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Modelo   : SVC
Acurácia : 47.07%
--------------------

tweets_mg
Modelo   : RandomForestClassifier
Acurácia : 52.39%
--------------------
Modelo   : LinearSVC
Acurácia : 53.05%
--------------------
Modelo   : MultinomialNB
Acurácia : 51.5%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 51.17%
--------------------
Modelo   : MLPClassifier
Acurácia : 53.5%
--------------------
Modelo   : SVC
Acurácia : 53.39%
--------------------


/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)



titulo_noticias
Modelo   : RandomForestClassifier
Acurácia : 45.37%
--------------------
Modelo   : LinearSVC
Acurácia : 43.17%
--------------------
Modelo   : MultinomialNB
Acurácia : 43.17%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 40.35%
--------------------
Modelo   : MLPClassifier
Acurácia : 43.33%
--------------------
Modelo   : SVC
Acurácia : 43.01%
--------------------


## Count

In [21]:
#all_datasets
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count = vec_count.fit_transform(afrases)

print("\nall_datasets")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_count, avalencias))
    except:
        pass

      
#tweets_mg
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tmg = vec_count.fit_transform(atweets_mg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_count_tmg, aval_tweets_mg))
    except:
        pass
      
      
#titulo_noticias
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tn = vec_count.fit_transform(atitulo_noticias)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_count_tn, aval_titulo_noticias))
    except:
        pass      


all_datasets
Modelo   : RandomForestClassifier
Acurácia : 63.91%
--------------------
Modelo   : LinearSVC
Acurácia : 65.41%
--------------------
Modelo   : MultinomialNB
Acurácia : 64.56%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 44.28%
--------------------
Modelo   : MLPClassifier
Acurácia : 64.24%
--------------------
Modelo   : SVC
Acurácia : 42.72%
--------------------

tweets_mg
Modelo   : RandomForestClassifier
Acurácia : 63.6%
--------------------
Modelo   : LinearSVC
Acurácia : 61.49%
--------------------
Modelo   : MultinomialNB
Acurácia : 62.15%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 47.84%
--------------------
Modelo   : MLPClassifier
Acurácia : 61.82%
--------------------
Modelo   : SVC
Acurácia : 43.51%
--------------------

titulo_noticias
Modelo   : RandomForestClassifier
Acurácia : 63.89%
--------------------
Modelo   : LinearSVC
Acurácia : 67.5%
--------------------
Modelo   : MultinomialNB
Acurácia : 66.09%
------------

## LSA (usando Count)

In [22]:
#all_datasets
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_svd, avalencias))
    except:
        pass
      

#tweets_mg
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
    except:
        pass
      
      
#titulos_noticias
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_count_tn)

print("\ntitulos_noticias")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
    except:
        pass      


all_datasets
Modelo   : RandomForestClassifier
Acurácia : 52.8%
--------------------
Modelo   : LinearSVC
Acurácia : 51.82%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 52.34%
--------------------
Modelo   : MLPClassifier
Acurácia : 52.54%
--------------------


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Modelo   : SVC
Acurácia : 52.28%
--------------------

tweets_mg
Modelo   : RandomForestClassifier
Acurácia : 57.27%
--------------------
Modelo   : LinearSVC
Acurácia : 61.04%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 56.94%
--------------------
Modelo   : MLPClassifier
Acurácia : 61.93%
--------------------


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Modelo   : SVC
Acurácia : 60.38%
--------------------

titulos_noticias
Modelo   : RandomForestClassifier
Acurácia : 50.39%
--------------------
Modelo   : LinearSVC
Acurácia : 48.35%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 48.51%
--------------------
Modelo   : MLPClassifier
Acurácia : 51.33%
--------------------
Modelo   : SVC
Acurácia : 48.35%
--------------------


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## LDA (usando Count)¶

In [23]:
#all_datasets
lda = LatentDirichletAllocation(n_components=5, random_state=0)
X_lda = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_lda, avalencias))
    except:
        pass


#tweets_mg
lda = LatentDirichletAllocation(n_components=5, random_state=0)
X_lda = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
    except:
        pass
      
      
#titulo_noticias
lda = LatentDirichletAllocation(n_components=5, random_state=0)
X_lda = lda.fit_transform(X_count_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
    except:
        pass

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)



all_datasets
Modelo   : RandomForestClassifier
Acurácia : 47.46%
--------------------
Modelo   : LinearSVC
Acurácia : 45.45%
--------------------
Modelo   : MultinomialNB
Acurácia : 44.86%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 46.1%
--------------------
Modelo   : MLPClassifier
Acurácia : 46.49%
--------------------


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Modelo   : SVC
Acurácia : 45.38%
--------------------

tweets_mg
Modelo   : RandomForestClassifier
Acurácia : 51.17%
--------------------
Modelo   : LinearSVC
Acurácia : 54.16%
--------------------
Modelo   : MultinomialNB
Acurácia : 53.83%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 51.39%
--------------------
Modelo   : MLPClassifier
Acurácia : 53.39%
--------------------
Modelo   : SVC
Acurácia : 53.72%
--------------------


/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)



titulo_noticias
Modelo   : RandomForestClassifier
Acurácia : 45.21%
--------------------
Modelo   : LinearSVC
Acurácia : 48.04%
--------------------
Modelo   : MultinomialNB
Acurácia : 48.19%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 42.39%
--------------------
Modelo   : MLPClassifier
Acurácia : 47.88%
--------------------
Modelo   : SVC
Acurácia : 47.1%
--------------------
